# Redes Neuronales Artificales

 **Instalar Theano:**
 
 * pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git
 
 
 **Instalar Tensorflow y Keras:**
 
 * conda install -c conda-forge keras
 

## Parte 1 - Pre procesado de datos

In [1]:
# Cómo importar las librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
# Importar el data set
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values #Si el cliente se queda en el banco o no
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Codificar datos categóricos (paises y generos) a variables numericas
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1]) #paises
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) #genero

In [7]:
# Ahora queremos transformar lso numeros asociados a los paises a variables dummie

#El OneHotEncoder en las nuevas versiones está OBSOLETO (justo debajo la forma de ahora)
#onehotencoder = OneHotEncoder(categorical_features=[1])
#X = onehotencoder.fit_transform(X).toarray()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",        # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]            # Las columnas a transformar.
         )
    ], remainder='passthrough'
)

X = transformer.fit_transform(X)
X = X[:, 1:]

In [8]:
# Dividir el data set en conjunto de entrenamiento y conjunto de testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# Escalado de variables
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Parte 2 - Construir la RNA

In [11]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential
from keras.layers import Dense #para crear las capas e incializar los pesos
from keras.layers import Dropout

Using TensorFlow backend.


In [12]:
# Inicializar la RNA
classifier = Sequential() #Sequential es la funcion para incializar las RNA

In [13]:
# Añadir las capas de entrada y primera capa oculta
#Dense se encarga de añadir propiedades a las conexiones de las capas. 
# -units es el numero de nodos que añadimos a la capa oculta(se suele poner la media entre las entradas y el nodo de salida (11+1)/2=6)
# -Kernel_initializer para elegir la funcion que usaran los pesos
# -activation es la funcion de activacion para activar en la capa oculta (relu=rectificador lineal unitario)
# -input_dim son los nodos de entrada
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11)) 
classifier.add(Dropout(p = 0.1)) #para evitar el overfitting

# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
classifier.add(Dropout(p = 0.1)) #para evitar el overfitting

# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
/home/david/.local/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  if sys.path[0] == '':


In [14]:
# Compilar la RNA
# -optimizer se encarga de buscar el conjunto optimo de pesos, usamos el algortimo adam
# -loss es la funcion de perdidas, la que minimiza el error
# -metrics es la metrica
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [15]:
# Ajustamos la RNA al Conjunto de Entrenamiento
# -batch_size numero de bloques, procesar dichos bloques y corregir
# -epochs es el numero de iteraciones
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 149us/step - loss: 0.4890 - accuracy: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 141us/step - loss: 0.4364 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 134us/step - loss: 0.4321 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4293 - accuracy: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 118us/step - loss: 0.4288 - accuracy: 0.8010
Epoch 6/100
8000/8000 [==============================] - 1s 142us/step - loss: 0.4229 - accuracy: 0.8255
Epoch 7/100
8000/8000 [==============================] - 1s 118us/step - loss: 0.4252 - accuracy: 0.8256
Epoch 8/100
8000/8000 [==============================] - 1s 152us/step - loss: 0.4268 - accuracy: 0.8253
Epoch 9/100
8000/8000 [==============================] - 1s 152us/step - loss: 0.4257 - accuracy: 0.8274
Epoch 10/100
8000/8000 [==============================]

## Parte 3 - Evaluar el modelo y calcular predicciones finales

In [17]:
# Predicción de los resultados con el Conjunto de Testing
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5) #los valores por encima de 0.5 seran considerados como que abandonan

In [18]:
# Elaborar una matriz de confusión
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print((cm[0][0]+cm[1][1])/cm.sum())

0.8395


## Parte 4 - Evaluar, mejorar y Ajustar la RNA

In [19]:
### Evaluar la **RNA** con K-Fold Validation en Keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier(): #Creamos las capas de la red neuronal
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform", activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform", activation = "sigmoid"))
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

mean = accuracies.mean()
variance = accuracies.std()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.0s finished


## Mejorar la RNA
Regularización de Dropout para evitar el *overfitting*

In [20]:
### Ajustar la *RNA*
from sklearn.model_selection import GridSearchCV # sklearn.grid_search en otra version antigua de Python

def build_classifier(optimizer):
  classifier = Sequential()
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu", input_dim = 11))
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])
  return classifier

classifier = KerasClassifier(build_fn = build_classifier)

parameters = {
    'batch_size' : [25,32],
    'nb_epoch' : [100, 500], 
    'optimizer' : ['adam', 'rmsprop']
}

grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_


Epoch 1/1
7200/7200 [==============================] - 0s 58us/step - loss: 0.5609 - accuracy: 0.7951
Epoch 1/1
7200/7200 [==============================] - 0s 66us/step - loss: 0.5651 - accuracy: 0.7943
Epoch 1/1
7200/7200 [==============================] - 0s 59us/step - loss: 0.5872 - accuracy: 0.7929
Epoch 1/1
7200/7200 [==============================] - 0s 60us/step - loss: 0.5917 - accuracy: 0.7950
Epoch 1/1
7200/7200 [==============================] - 0s 58us/step - loss: 0.5545 - accuracy: 0.7928
Epoch 1/1
7200/7200 [==============================] - 0s 62us/step - loss: 0.5671 - accuracy: 0.7931
Epoch 1/1
7200/7200 [==============================] - 1s 74us/step - loss: 0.5746 - accuracy: 0.7960
Epoch 1/1
7200/7200 [==============================] - 0s 68us/step - loss: 0.5520 - accuracy: 0.7949
Epoch 1/1
7200/7200 [==============================] - 1s 86us/step - loss: 0.5732 - accuracy: 0.7944
Epoch 1/1
7200/7200 [==============================] - 1s 72us/step - loss: 0.5650